In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-optimize

### import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns

### Data Load

In [ ]:
# 학습/평가 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/용훈/Dacon/채무 불이행 확률 예측/open/train.csv').drop(columns=['UID'])
test_df = pd.read_csv('/content/drive/MyDrive/용훈/Dacon/채무 불이행 확률 예측/open/test.csv').drop(columns=['UID'])

In [ ]:
train_df

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,주택 담보 대출 (거주 중),1339473.0,10년 이상,0.0,9,18.7,319027.5,0,68,0,부채 통합,단기 상환,126216.0,177028.5,6237.0,755,0
9996,주택 담보 대출 (거주 중),2297230.5,2년,0.0,11,28.3,399799.5,0,7,0,주택 개보수,장기 상환,371907.0,347449.5,53301.0,707,0
9997,주택 담보 대출 (거주 중),1221523.5,10년 이상,0.0,9,30.1,823305.0,0,14,0,부채 통합,장기 상환,869736.0,176905.5,11436.0,733,0
9998,자가,3343584.0,10년 이상,0.0,10,20.3,724314.0,0,25,0,부채 통합,단기 상환,443008.5,139294.5,25567.5,696,0


In [ ]:
train_df.dtypes

,0
주거 형태,object
연간 소득,float64
현재 직장 근속 연수,object
체납 세금 압류 횟수,float64
개설된 신용계좌 수,int64
신용 거래 연수,float64
최대 신용한도,float64
신용 문제 발생 횟수,int64
마지막 연체 이후 경과 개월 수,int64
개인 파산 횟수,int64


In [ ]:
# 결측치 없음
# print(train_df.isna().sum())
# print(test_df.isna().sum())

- 연간 소득의 경우 매우매우 많은 이상치가 존재 / 연간 소득 RobustScaler

In [ ]:
# plt.boxplot(train_df['연간 소득'], vert=False)

# plt.show

# print(train_df['연간 소득'].min())
# print(train_df['연간 소득'].mean())
# print(train_df['연간 소득'].median())
# print(train_df['연간 소득'].max())

In [ ]:
plt.rc('font', family='Malgun Gothic')
import math

# 수치형 컬럼 선택
num_cols = train_df.select_dtypes(include=['number']).columns

# # 서브플롯 크기 조절
# num_features = len(num_cols)
# cols = 3  # 한 행에 표시할 컬럼 수
# rows = math.ceil(num_features / cols)  # 필요한 행 수 계산

# # 그래프 그리기
# fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 4))  # 전체 크기 조절
# axes = axes.flatten()  # 2D 배열을 1D로 변환

# for i, col in enumerate(num_cols):
#     sns.histplot(train_df[col], kde=True, ax=axes[i])  # 히스토그램 + KDE 추가
#     axes[i].set_title(col)  # 제목 설정

# # 빈 subplot 숨기기
# for i in range(num_features, len(axes)):
#     fig.delaxes(axes[i])

# plt.tight_layout()  # 간격 조정
# plt.show()


### Pre-processing(전처리)

In [ ]:
categorical_col = [
    '주거 형태',
    '현재 직장 근속 연수',
    '대출 목적',
    '대출 상환 기간'
]

train_df = pd.get_dummies(train_df, columns=categorical_col, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_col, drop_first=True)

train_df

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,...,대출 목적_사업 대출,대출 목적_소규모 사업 자금,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환
0,1941337.5,0.0,9,13.4,400597.5,0,24,1,390903.0,225457.5,...,False,False,False,False,False,False,False,False,False,False
1,1979505.0,0.0,5,15.1,360679.5,0,11,0,1002184.5,64749.0,...,False,False,False,False,False,False,False,False,False,False
2,1356381.0,0.0,12,18.8,491770.5,1,74,3,227775.0,487644.0,...,False,False,False,False,False,False,False,False,False,False
3,1049017.5,0.0,15,14.8,411546.0,1,22,1,251383.5,413211.0,...,False,False,False,False,False,False,False,False,False,False
4,4320217.5,0.0,11,26.1,895288.5,0,32,0,1163176.5,78991.5,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1339473.0,0.0,9,18.7,319027.5,0,68,0,126216.0,177028.5,...,False,False,False,False,False,False,False,False,False,False
9996,2297230.5,0.0,11,28.3,399799.5,0,7,0,371907.0,347449.5,...,False,False,False,False,False,False,True,False,False,True
9997,1221523.5,0.0,9,30.1,823305.0,0,14,0,869736.0,176905.5,...,False,False,False,False,False,False,False,False,False,True
9998,3343584.0,0.0,10,20.3,724314.0,0,25,0,443008.5,139294.5,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
num_cols = num_cols.drop('채무 불이행 여부')
num_cols

Index(['연간 소득', '체납 세금 압류 횟수', '개설된 신용계좌 수', '신용 거래 연수', '최대 신용한도',
       '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '개인 파산 횟수', '현재 대출 잔액',
       '현재 미상환 신용액', '월 상환 부채액', '신용 점수'],
      dtype='object')

### Train / Validation Split (학습 데이터 분할)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop(columns=['채무 불이행 여부']),
    train_df['채무 불이행 여부'],
    test_size=0.2,
    random_state=42
)

### StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_val_scaled = scaler.transform(X_val[num_cols])

# 2. 정규화 결과를 DataFrame으로 변환
X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_cols, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=num_cols, index=X_val.index)

# 3. 원-핫 인코딩된 범주형 컬럼 추가
X_train_final = pd.concat([X_train_scaled, X_train.drop(columns=num_cols)], axis=1)
X_val_final = pd.concat([X_val_scaled, X_val.drop(columns=num_cols)], axis=1)

# 테스트 데이터도 동일한 방식으로 처리
test_df_scaled = scaler.transform(test_df[num_cols])
test_df_scaled = pd.DataFrame(test_df_scaled, columns=num_cols, index=test_df.index)
test_df_final = pd.concat([test_df_scaled, test_df.drop(columns=num_cols)], axis=1)

### MinMaxScaler

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

# X_train_scaled = scaler.fit_transform(X_train[num_cols])
# X_val_scaled = scaler.transform(X_val[num_cols])

# # 2. 정규화 결과를 DataFrame으로 변환
# X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_cols, index=X_train.index)
# X_val_scaled = pd.DataFrame(X_val_scaled, columns=num_cols, index=X_val.index)

# # 3. 원-핫 인코딩된 범주형 컬럼 추가
# X_train_final = pd.concat([X_train_scaled, X_train.drop(columns=num_cols)], axis=1)
# X_val_final = pd.concat([X_val_scaled, X_val.drop(columns=num_cols)], axis=1)

# # 테스트 데이터도 동일한 방식으로 처리
# test_df_scaled = scaler.transform(test_df[num_cols])
# test_df_scaled = pd.DataFrame(test_df_scaled, columns=num_cols, index=test_df.index)
# test_df_final = pd.concat([test_df_scaled, test_df.drop(columns=num_cols)], axis=1)

### RobustScaler

In [ ]:
# from sklearn.preprocessing import RobustScaler

# scaler = RobustScaler()

# X_train_scaled = scaler.fit_transform(X_train[num_cols])
# X_val_scaled = scaler.transform(X_val[num_cols])

# # 2. 정규화 결과를 DataFrame으로 변환
# X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_cols, index=X_train.index)
# X_val_scaled = pd.DataFrame(X_val_scaled, columns=num_cols, index=X_val.index)

# # 3. 원-핫 인코딩된 범주형 컬럼 추가
# X_train_final = pd.concat([X_train_scaled, X_train.drop(columns=num_cols)], axis=1)
# X_val_final = pd.concat([X_val_scaled, X_val.drop(columns=num_cols)], axis=1)

# # 테스트 데이터도 동일한 방식으로 처리
# test_df_scaled = scaler.transform(test_df[num_cols])
# test_df_scaled = pd.DataFrame(test_df_scaled, columns=num_cols, index=test_df.index)
# test_df_final = pd.concat([test_df_scaled, test_df.drop(columns=num_cols)], axis=1)

In [ ]:
# # 불균형 데이터 처리
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_train_final, y_train = smote.fit_resample(X_train_final, y_train)

# print("\nResampled class distribution:")
# print(pd.Series(y_train).value_counts())

### Model Training (모델 학습)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

### 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()

model_logistic.fit(X_train_final, y_train)

y_pred_logistic = model_logistic.predict(X_val_final)

print(roc_auc_score(y_val, y_pred_logistic))
print(classification_report(y_val, y_pred_logistic))

0.6262700895264804
              precision    recall  f1-score   support

           0       0.71      0.88      0.79      1270
           1       0.64      0.37      0.47       730

    accuracy                           0.70      2000
   macro avg       0.68      0.63      0.63      2000
weighted avg       0.69      0.70      0.67      2000



### SVC

In [ ]:
from sklearn.svm import SVC

model_svc = SVC(probability=True)

model_svc.fit(X_train_final, y_train)

y_pred_svc = model_svc.predict(X_val_final)
y_pred_proba_scv = model_svc.predict_proba(X_val_final)[:, 1]

print(roc_auc_score(y_val, y_pred_proba_scv))
print(classification_report(y_val, y_pred_svc))

0.7190357027289397
              precision    recall  f1-score   support

           0       0.71      0.90      0.79      1270
           1       0.67      0.34      0.45       730

    accuracy                           0.70      2000
   macro avg       0.69      0.62      0.62      2000
weighted avg       0.69      0.70      0.67      2000



### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(
    n_estimators=100,  # 트리 개수
    max_depth=30,      # 최대 깊이
    random_state=42,
    n_jobs=-1,         # 병렬 처리
)

model_RF.fit(X_train_final, y_train)

y_pred_RF = model_RF.predict(X_val_final)
y_pred_proba_RF = model_RF.predict_proba(X_val_final)[:, 1]

print(roc_auc_score(y_val, y_pred_proba_RF))
print(classification_report(y_val, y_pred_RF))

0.7264318843706182
              precision    recall  f1-score   support

           0       0.71      0.90      0.80      1270
           1       0.68      0.37      0.48       730

    accuracy                           0.71      2000
   macro avg       0.70      0.64      0.64      2000
weighted avg       0.70      0.71      0.68      2000



### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

model_LGBM = LGBMClassifier(
    n_estimators=100,   # 트리 개수
    max_depth=30,       # 최대 깊이
    learning_rate=0.1,  # 학습률
    random_state=42,
    objective="binary", # 이진 분류
)

model_LGBM.fit(
    X_train_final, y_train,
    eval_set=[(X_val_final, y_val)],
    eval_metric="auc",
)

y_pred_LGBM = model_LGBM.predict(X_val_final)
y_pred_proba_LGBM = model_LGBM.predict_proba(X_val_final)[:, 1]

print(roc_auc_score(y_val, y_pred_proba_LGBM))
print(classification_report(y_val, y_pred_LGBM))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2682, number of negative: 5318
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1850
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 35
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335250 -> initscore=-0.684535
[LightGBM] [Info] Start training from score -0.684535
0.7278211627656133
              precision    recall  f1-score   support

           0       0.72      0.87      0.79      1270
           1       0.64      0.42      0.51       730

    accuracy                           0.70      2000
   macro avg       0.68      0.64      0.65     

### XGB

In [ ]:
from xgboost import XGBClassifier
# XGBoost 모델 학습
model_XGB = XGBClassifier(
    n_estimators=100,  # 트리 개수
    max_depth=30,       # 최대 깊이
    learning_rate=0.1, # 학습률
    random_state=42,
    use_label_encoder=False,  # 경고 방지
    eval_metric="auc",        # 평가 지표 설정
)

# 학습 및 Validation 성능 모니터링
eval_set = [(X_train_final, y_train), (X_val_final, y_val)]
model_XGB.fit(
    X_train_final, y_train,
    eval_set=eval_set,
    verbose=False,
)

y_pred_XGB = model_XGB.predict(X_val_final)
y_pred_proba_XGB = model_XGB.predict_proba(X_val_final)[:, 1]

print(roc_auc_score(y_val, y_pred_proba_XGB))
print(classification_report(y_val, y_pred_XGB))

0.7058095135368353
              precision    recall  f1-score   support

           0       0.71      0.85      0.78      1270
           1       0.62      0.41      0.49       730

    accuracy                           0.69      2000
   macro avg       0.67      0.63      0.63      2000
weighted avg       0.68      0.69      0.67      2000



### CatBoostClassifier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

# CatBoost 모델 초기화
model_CB = CatBoostClassifier(
    iterations=300,  # 트리 개수 (XGBoost의 n_estimators와 유사)
    depth=7,        # 최대 깊이
    learning_rate=0.03,  # 학습률
    random_state=42,
    eval_metric='AUC',  # 평가 지표 설정
    verbose=False,  # 학습 과정 출력 여부
)

# 학습 및 Validation 성능 모니터링
eval_set = Pool(X_val_final, label=y_val)  # 평가 데이터셋
model_CB.fit(
    X_train_final, y_train,
    eval_set=eval_set,
    early_stopping_rounds=30,
    verbose=False,
    use_best_model=True
)

# 예측 및 평가
y_pred_CB = model_CB.predict(X_val_final)
y_pred_proba_CB = model_CB.predict_proba(X_val_final)[:, 1]

print(roc_auc_score(y_val, y_pred_proba_CB))
print(classification_report(y_val, y_pred_CB))


0.7439715241074318
              precision    recall  f1-score   support

           0       0.72      0.90      0.80      1270
           1       0.69      0.38      0.49       730

    accuracy                           0.71      2000
   macro avg       0.70      0.64      0.64      2000
weighted avg       0.71      0.71      0.69      2000



### RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 하이퍼파라미터 범위 (랜덤 선택)
param_dist = {
    "n_estimators": np.random.randint(50, 300, 10).tolist(),  # 50~300 사이 랜덤한 5개 값
    "max_depth": np.random.randint(3, 20, 10).tolist(),  # 3~20 사이 랜덤한 5개 값
    "learning_rate": np.random.uniform(0.01, 0.3, 10).tolist(),  # 0.01~0.3 사이 랜덤한 5개 값
    "subsample": np.random.uniform(0.5, 1.0, 10).tolist(),  # 0.5~1.0 사이 랜덤한 5개 값
    # "colsample_bytree": np.random.uniform(0.5, 1.0, 10).tolist()  # 0.5~1.0 사이 랜덤한 5개 값
}

# RandomizedSearchCV 함수
def RCV(model):
    print('-----' * 20)
    print(f'{model}')

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,  # 랜덤 샘플링된 파라미터 사용
        n_iter=10,  # 10번만 랜덤 탐색
        scoring="roc_auc",
        cv=5,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # 학습 실행
    random_search.fit(X_train_final, y_train)

    # 최적 하이퍼파라미터 출력
    print("🔥 Best Parameters:", random_search.best_params_)
    print("🔥 Best Score:", random_search.best_score_)

    # 예측 및 평가
    y_pred = random_search.predict(X_val_final)
    y_pred_proba = random_search.predict(X_val_final)

    print(roc_auc_score(y_val, y_pred_proba))
    print(classification_report(y_val, y_pred))

    return random_search

### GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
        "n_estimators": [100, 200],   # 트리 개수
        "max_depth": range(3, 15),      # 트리 최대 깊이
        "learning_rate": [0.01, 0.1, 0.2],  # 학습률
        "subsample": [0.7, 1.0],      # 샘플링 비율
        # "colsample_bytree": [0.7, 1.0] # 트리별 사용할 feature 비율
    }

def GCV(model) :
    print('-----' * 30)
    print(f'{model}')

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring="roc_auc",  # AUC 점수 기준 최적화
        cv=3,  # 5-Fold Cross Validation
        verbose=0,
        n_jobs=-1  # 병렬 처리
    )

    # 학습 실행
    grid_search.fit(X_train_final, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Parameters:", grid_search.best_params_)
    print("Best Score:", grid_search.best_score_)

    y_pred = grid_search.predict(X_val_final)
    y_pred_proba = grid_search.predict_proba(X_val_final)[:, 1]

    print(roc_auc_score(y_val, y_pred_proba))
    print(classification_report(y_val, y_pred))

    return grid_search

### BayesSearchCV

In [ ]:
from skopt import BayesSearchCV

param_space = {
        'max_depth': (3, 15),
        'learning_rate': (0.01, 0.3),
        'n_estimators': (50, 200),
        'subsample': (0.5, 1),
        # 'colsample_bytree': (0.5, 1)
    }

def BCV(model) :
    # 기본 XGB
    print('-----' * 30)
    print(f'{model}')

    opt = BayesSearchCV(model_XGB, param_space, n_iter=50, cv=5)
    opt.fit(X_train_final, y_train)

    print(opt.best_params_)

    y_pred = opt.predict(X_val_final)
    y_pred_proba = opt.predict_proba(X_val_final)[:, 1]

    print(f'{model}')
    print(roc_auc_score(y_val, y_pred_proba))
    print(classification_report(y_val, y_pred))

    return opt

In [ ]:
# RCV_RFC = RCV(RandomForestClassifier())
# RCV_LGB = RCV(LGBMClassifier())
# RCV_XGB = RCV(XGBClassifier())
# RCV_CB = RCV(CatBoostClassifier())

In [ ]:
# GCV_RFC = GCV(RandomForestClassifier())
GCV_LGB = GCV(LGBMClassifier())
GCV_XGB = GCV(XGBClassifier())
# GCV_CB = RCV(CatBoostClassifier())

------------------------------------------------------------------------------------------------------------------------------------------------------
LGBMClassifier()
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2682, number of negative: 5318
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1850
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335250 -> initscore=-0.684535
[LightGBM] [Info] Start training from score -0.684535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

In [ ]:
# BCV_RFC = BCV(RandomForestClassifier())
BCV_LGB = BCV(LGBMClassifier())
BCV_XGB = BCV(XGBClassifier())
# BCV_CB = RCV(CatBoostClassifier())

------------------------------------------------------------------------------------------------------------------------------------------------------
LGBMClassifier()
OrderedDict([('learning_rate', 0.1015846708541794), ('max_depth', 3), ('n_estimators', 117), ('subsample', 0.5)])
LGBMClassifier()
0.7439758386366089
              precision    recall  f1-score   support

           0       0.72      0.88      0.80      1270
           1       0.67      0.41      0.51       730

    accuracy                           0.71      2000
   macro avg       0.70      0.65      0.65      2000
weighted avg       0.70      0.71      0.69      2000

------------------------------------------------------------------------------------------------------------------------------------------------------
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
    

### 기여도에 따른 변수 선택

In [ ]:
# SelectFromModel을 사용하여 중요한 특징 선택
from sklearn.feature_selection import SelectFromModel

def select_feature(model, threshold = None) :
    # 특징 중요도 얻기
    feature_importance = model.feature_importances_

    original_feature_names = X_train_final.columns

    # 특징 이름과 중요도를 튜플로 묶기
    feature_importance_pairs = list(zip(original_feature_names, feature_importance))

    # 중요도에 따라 정렬
    sorted_features = sorted(feature_importance_pairs, key=lambda x: x[1], reverse=True)

    # 정렬된 특징 중요도 출력
    print("특징 중요도 (내림차순):")
    for feature, importance in sorted_features:
        print(f"{feature}: {importance:.4f}")

    ########################################################################################################################
    # 자동 특징 선택
    #selection = SelectFromModel(model, prefit=True)

    # threshold값 이상 특징 선택
    selection = SelectFromModel(model, threshold=threshold, prefit=True)

    # 또는 상위 10개 특징 선택
    # selection = SelectFromModel(model, max_features=30, prefit=True)
    ########################################################################################################################

    selected_features = selection.get_support()
    print(selected_features)

    selected_feature_names = original_feature_names[selected_features]
    print(len(selected_feature_names), '/', len(original_feature_names))
    print("선택된 특징:", selected_feature_names)


    return selection

In [ ]:
# X_train_selected = selection.transform(X_train)
# X_test_selected = selection.transform(test_df_final)

# # 선택된 특징으로 새로운 모델 학습
# selected_model = XGBClassifier(
#     n_estimators=100,  # 트리 개수
#     max_depth=30,       # 최대 깊이
#     learning_rate=0.1, # 학습률
#     random_state=42,
#     use_label_encoder=False,  # 경고 방지
#     eval_metric="auc",        # 평가 지표 설정
# )

# selected_model.fit(X_train_selected, y_train)

# selection XGB
# param_space = {
#     'max_depth': (3, 10),
#     'learning_rate': (0.01, 0.3),
#     'n_estimators': (50, 200),
#     'subsample': (0.5, 1),
#     'colsample_bytree': (0.5, 1)
# }

# opt = BayesSearchCV(selected_model, param_space, n_iter=50, cv=5)
# opt.fit(X_train_selected, y_train)
# print(opt.best_params_)

In [ ]:
# 하이퍼파라미터 그리드 설정
param_grid = {
        "n_estimators": [100, 200],   # 트리 개수
        "max_depth": range(3, 15),      # 트리 최대 깊이
        "learning_rate": [0.01, 0.1, 0.2],  # 학습률
        "subsample": [0.7, 1.0],      # 샘플링 비율
        # "colsample_bytree": [0.7, 1.0] # 트리별 사용할 feature 비율
    }

def GCV_select(model, xt, yt, xv, yv) :
    print('-----' * 30)
    print(f'{model}')

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring="roc_auc",  # AUC 점수 기준 최적화
        cv=3,  # 5-Fold Cross Validation
        verbose=0,
        n_jobs=-1  # 병렬 처리
    )

    # 학습 실행
    grid_search.fit(xt, yt)

    # 최적 하이퍼파라미터 출력
    print("Best Parameters:", grid_search.best_params_)
    print("Best Score:", grid_search.best_score_)

    y_pred = grid_search.predict(xv)
    y_pred_proba = grid_search.predict_proba(xv)[:, 1]

    print(roc_auc_score(yv, y_pred_proba))
    print(classification_report(yv, y_pred))

    return grid_search

In [ ]:
param_space = {
        'max_depth': (3, 15),
        'learning_rate': (0.01, 0.3),
        'n_estimators': (50, 200),
        'subsample': (0.5, 1),
        # 'colsample_bytree': (0.5, 1)
    }

def BCV_select(model, xt, yt, xv, yv) :
    # 기본 XGB
    print('-----' * 30)
    print(f'{model}')

    opt = BayesSearchCV(model_XGB, param_space, n_iter=50, cv=5)
    opt.fit(xt, yt)

    print(opt.best_params_)

    y_pred = opt.predict(xv)
    y_pred_proba = opt.predict_proba(xv)[:, 1]

    print(f'{model}')
    print(roc_auc_score(yv, y_pred_proba))
    print(classification_report(yv, y_pred))

    return opt

In [ ]:
# selection = select_feature(BCV_LGB, threshold=0.01)
# selection = select_feature(BCV_XGB, threshold=0.01)
selection = select_feature(model_CB, threshold=0.1)

X_train_selected = selection.transform(X_train)
X_val_selected = selection.transform(X_val_final)
X_test_selected = selection.transform(test_df_final)

# GCV_select(LGBMClassifier(), X_train_selected, y_train, X_val_final, y_val)
BCV_LGB_SELECT = BCV_select(BCV_LGB, X_train_selected, y_train, X_val_selected, y_val)
# GCV_select(XGBClassifier(), X_train_selected, y_train, X_val_final, y_val)
BCV_XGB_SELECT = BCV_select(BCV_XGB, X_train_selected, y_train, X_val_selected, y_val)

특징 중요도 (내림차순):
개설된 신용계좌 수: 14.4072
연간 소득: 11.1188
현재 대출 잔액: 8.8929
대출 목적_부채 통합: 8.5331
신용 점수: 8.0162
현재 미상환 신용액: 7.7573
대출 상환 기간_장기 상환: 6.1019
신용 거래 연수: 5.9519
신용 문제 발생 횟수: 5.4415
마지막 연체 이후 경과 개월 수: 4.1632
최대 신용한도: 3.5331
개인 파산 횟수: 3.4663
월 상환 부채액: 3.4457
현재 직장 근속 연수_2년: 2.2196
체납 세금 압류 횟수: 1.4254
현재 직장 근속 연수_1년: 1.0637
주거 형태_자가: 1.0060
주거 형태_주택 담보 대출 (거주 중): 0.6865
현재 직장 근속 연수_8년: 0.6292
대출 목적_주택 개보수: 0.4913
현재 직장 근속 연수_3년: 0.2688
주거 형태_주택 담보 대출 (비거주 중): 0.2447
현재 직장 근속 연수_5년: 0.2416
현재 직장 근속 연수_9년: 0.1461
대출 목적_의료비: 0.1417
현재 직장 근속 연수_7년: 0.0980
현재 직장 근속 연수_1년 미만: 0.0971
현재 직장 근속 연수_4년: 0.0819
대출 목적_기타: 0.0734
현재 직장 근속 연수_6년: 0.0686
대출 목적_여행 자금: 0.0490
대출 목적_소규모 사업 자금: 0.0461
대출 목적_사업 대출: 0.0394
대출 목적_자동차 구매: 0.0278
대출 목적_고액 구매: 0.0179
대출 목적_주택 구매: 0.0071
대출 목적_교육비: 0.0000
대출 목적_이사 비용: 0.0000
대출 목적_휴가 비용: 0.0000
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True False  True False False  True
  True False False False  Tr

### Prediction

In [ ]:
# preds = opt.predict_proba(test_df_final)[:,1]
# preds

# preds_BCV_RFC = BCV_RFC.predict_proba(test_df_final)[:,1]
preds_BCV_LGB = BCV_LGB.predict_proba(test_df_final)[:,1]
preds_BCV_XGB = BCV_XGB.predict_proba(test_df_final)[:,1]
preds_CB = model_CB.predict_proba(test_df_final)[:,1]
preds_BCV_LGB_SELECT = BCV_LGB_SELECT.predict_proba(X_test_selected)[:,1]
preds_BCV_XGB_SELECT = BCV_XGB_SELECT.predict_proba(X_test_selected)[:,1]
# preds_BCV_CB = BCV_CB.predict_proba(test_df_final)[:,1]

In [ ]:
def predict_proba(model, test) :
    return model.predict_proba(test_df_final)[:,1]

In [ ]:
# # 채무 불이행 '확률'을 예측합니다.
# preds = opt.predict_proba(X_test_selected)[:,1]
# preds

### Submission (제출 파일 생성)

In [ ]:
# def predict_proba(model) :
#     return model.predict_proba(test_df_final)[:,1]

# def predict_proba(model) :
#     return model.predict_proba(X_test_selected)[:,1]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/용훈/Dacon/채무 불이행 확률 예측/open/sample_submission.csv')

# 결과 저장
# submit['채무 불이행 확률'] = preds_BCV_RFC
# submit.to_csv('./submissionRFC.csv', encoding='UTF-8-sig', index=False)

submit['채무 불이행 확률'] = preds_BCV_LGB
submit.to_csv('./submissionLGB.csv', encoding='UTF-8-sig', index=False)

submit['채무 불이행 확률'] = preds_BCV_XGB
submit.to_csv('./submissionXGB.csv', encoding='UTF-8-sig', index=False)

submit['채무 불이행 확률'] = preds_CB
submit.to_csv('./submissionCB_SELECT.csv', encoding='UTF-8-sig', index=False)

submit['채무 불이행 확률'] = preds_BCV_LGB_SELECT
submit.to_csv('./submissionLGB_SELECT.csv', encoding='UTF-8-sig', index=False)

submit['채무 불이행 확률'] = preds_BCV_XGB_SELECT
submit.to_csv('./submissionXGB_SELECT.csv', encoding='UTF-8-sig', index=False)



# submit['채무 불이행 확률'] = preds_BCV_CB
# submit.to_csv('./submissionCB.csv', encoding='UTF-8-sig', index=False)